In [4]:
import pandas as pd
import requests
import time
from tqdm import tqdm



In [5]:
dl_all_df = pd.read_csv('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/all_delistings_since_2020.csv')


In [6]:
print(len(dl_all_df))
print(dl_all_df.columns)
display(dl_all_df.head())

574
Index(['date', 'company', 'cik', 'form'], dtype='object')


,date,company,cik,form
0,2024-10-31,"Finch Therapeutics Group, Inc.",1733257,25
1,2024-10-21,TELLURIAN INC. /DE/,61398,25
2,2024-10-11,"Guardion Health Sciences, Inc.",1642375,25
3,2024-10-11,Investment Managers Series Trust II,1587982,25
4,2024-10-07,ORANGE,1038143,25


In [21]:
missing_cik = dl_all_df[dl_all_df['cik'].isnull()]
print(f"Number of rows with missing CIK: {len(missing_cik)}")
display(missing_cik)

Number of rows with missing CIK: 0


,date,company,cik,form


In [10]:
# def get_sec_company_info(cik):
#     """
#     Get only ticker and industry info from SEC using CIK
#     """
#     try:
#         headers = {
#             'User-Agent': 'sanspeursystems matt@sanspeursystems.com',  # Replace with your details
#             'Accept-Encoding': 'gzip, deflate'
#         }
        
#         url = f"https://data.sec.gov/submissions/CIK{str(cik).zfill(10)}.json"
#         response = requests.get(url, headers=headers)
        
#         if response.status_code == 200:
#             data = response.json()
#             return {
#                 'ticker': data.get('tickers', [None])[0],
#                 'sic_description': data.get('sicDescription')
#             }
#         return {'ticker': None, 'sic_description': None}
#     except Exception as e:
#         return {'ticker': None, 'sic_description': None}

# def create_delisting_summary(df):
#     """
#     Create a clean DataFrame with just company name, date, ticker, and industry
#     """
#     # Initialize new DataFrame with essential columns
#     dl_summary_df = pd.DataFrame(columns=['date', 'company', 'cik', 'ticker', 'industry'])
    
#     # Process each company
#     print("Fetching company information...")
#     rows = []
#     for _, row in tqdm(df.iterrows(), total=len(df)):
#         info = get_sec_company_info(row['cik'])
        
#         rows.append({
#             'date': row['date'],
#             'company': row['company'],
#             'cik': row['cik'],
#             'ticker': info['ticker'],
#             'industry': info['sic_description']
#         })
        
#         # SEC API rate limit compliance
#         time.sleep(0.1)
    
#     # Create new DataFrame from collected data
#     dl_summary_df = pd.DataFrame(rows)
    
#     # Clean up the data
#     dl_summary_df['date'] = pd.to_datetime(dl_summary_df['date'])
#     dl_summary_df = dl_summary_df.sort_values('date', ascending=False)
    
#     return dl_summary_df

In [11]:
# # Create the simplified delisting summary
# delisting_summary = create_delisting_summary(dl_all_df)

# # Display results
# print("\nSample of Delisting Summary:")
# print(delisting_summary.head())

# # Basic statistics
# print("\nSummary Statistics:")
# print(f"Total delistings: {len(delisting_summary)}")
# print(f"Companies with identified tickers: {delisting_summary['ticker'].notna().sum()}")
# print(f"Companies with identified industries: {delisting_summary['industry'].notna().sum()}")

# # Save to CSV
# delisting_summary.to_csv('delisting_summary.csv', index=False)

Fetching company information...


100%|██████████| 574/574 [04:31<00:00,  2.11it/s]


Sample of Delisting Summary:
        date                              company      cik ticker  \
0 2024-10-31       Finch Therapeutics Group, Inc.  1733257   FNCH   
1 2024-10-21                  TELLURIAN INC. /DE/    61398   TELZ   
2 2024-10-11       Guardion Health Sciences, Inc.  1642375   GHSI   
3 2024-10-11  Investment Managers Series Trust II  1587982   None   
4 2024-10-07                               ORANGE  1038143  FNCTF   

                                          industry  
0  Biological Products, (No Diagnostic Substances)  
1                    Crude Petroleum & Natural Gas  
2                      Pharmaceutical Preparations  
3                                             None  
4     Telephone Communications (No Radiotelephone)  

Summary Statistics:
Total delistings: 574
Companies with identified tickers: 300
Companies with identified industries: 300


# 4

In [11]:
import pandas as pd
import io
import requests
import time
from datetime import datetime
from tqdm import tqdm

In [15]:
def read_delisting_data(csv_text):
    """
    Reads delisting data from CSV text and returns a DataFrame
    """
    # Convert the CSV text to DataFrame
    df = pd.read_csv(pd.StringIO(csv_text))
    
    # Clean up CIK numbers (ensure 10 digits with leading zeros)
    df['cik'] = df['cik'].astype(str).str.zfill(10)
    
    # Convert date string to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    return df

def get_company_info(cik, headers):
    """
    Get company SIC code and other info from SEC
    """
    # Format CIK to 10 digits
    cik_padded = str(cik).zfill(10)
    url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            tickers = data.get('tickers', [])
            ticker = tickers[0] if tickers else ''
            return {
                'sic': data.get('sicCode', ''),
                'sic_description': data.get('sicDescription', ''),
                'ticker': ticker
            }
    except Exception as e:
        print(f"Error fetching company info for CIK {cik}: {str(e)}")
    return None

def process_delistings(df):
    """
    Process delisting data and add ticker symbols and SIC information
    """
    # Add new columns
    df['ticker'] = None
    df['sic'] = None
    df['sic_description'] = None
    
    # Process each company
    for idx, row in df.iterrows():
        company_info = get_company_info(row['cik'])
        if company_info:
            df.at[idx, 'ticker'] = company_info['ticker']
            df.at[idx, 'sic'] = company_info['sic']
            df.at[idx, 'sic_description'] = company_info['sic_description']
        time.sleep(0.1)  # Respect SEC API rate limits
    
    # Filter for biotech companies (SIC code 2836)
    biotech_companies = df[df['sic'] == '2836'].copy()
    
    return df, biotech_companies

def export_results(df, biotech_df, output_file):
    """
    Export results to Excel with multiple sheets
    """
    with pd.ExcelWriter(output_file) as writer:
        df.to_excel(writer, sheet_name='All Delistings', index=False)
        biotech_df.to_excel(writer, sheet_name='Biotech Companies', index=False)

In [16]:
# Read the input data
dee_l_df = read_delisting_data('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/all_delistings_since_2020.csv')

# dee_l_df = read_delisting_data("""date,company,cik,form
# [Your CSV data here]""")

# Process the data
processed_df, biotech_df = process_delistings(dee_l_df)

# Export results
export_results(processed_df, biotech_df, 'delisting_analysis.xlsx')

# Print summary statistics
print(f"Total delistings: {len(processed_df)}")
print(f"Biotech companies: {len(biotech_df)}")
print(f"Percentage biotech: {(len(biotech_df) / len(processed_df)) * 100:.1f}%")

AttributeError: module 'pandas' has no attribute 'StringIO'

In [ ]:
def get_company_info(cik, headers):
    """
    Get company SIC code and other info from SEC
    """
    # Format CIK to 10 digits
    cik_padded = str(cik).zfill(10)
    url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            tickers = data.get('tickers', [])
            ticker = tickers[0] if tickers else ''
            return {
                'sic': data.get('sicCode', ''),
                'sic_description': data.get('sicDescription', ''),
                'ticker': ticker
            }
    except Exception as e:
        print(f"Error fetching company info for CIK {cik}: {str(e)}")
    return None

In [17]:
import pandas as pd
from io import StringIO

def read_delisting_data(file_path):
    """
    Reads delisting data from a CSV file and returns a DataFrame
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Clean up CIK numbers (ensure 10 digits with leading zeros)
    df['cik'] = df['cik'].astype(str).str.zfill(10)
    return df

# Read the input data
dee_l_df = read_delisting_data('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/all_delistings_since_2020.csv')

# Process the data
processed_df, biotech_df = process_delistings(dee_l_df)

# Export results
export_results(processed_df, biotech_df, 'delisting_analysis.xlsx')

# Print summary statistics
print(f"Total delistings: {len(processed_df)}")
print(f"Biotech companies: {len(biotech_df)}")
print(f"Percentage biotech: {(len(biotech_df) / len(processed_df)) * 100:.1f}%")

TypeError: get_company_info() missing 1 required positional argument: 'headers'

# GET IT 4 AGAIN part deux

In [ ]:
import pandas as pd
import requests
import time
from io import StringIO

def read_delisting_data(file_path):
    """
    Reads delisting data from a CSV file and returns a DataFrame
    """
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Clean up CIK numbers (ensure 10 digits with leading zeros)
    df['cik'] = df['cik'].astype(str).str.zfill(10)
    
    # Convert date string to datetime
    df['date'] = pd.to_datetime(df['date'])
    
    return df

def get_company_info(cik):
    """
    Get company SIC code and other info from SEC
    """
    headers = {
        'User-Agent': 'sanspeursystems matt@sanspeursystems.com',
        'Accept-Encoding': 'gzip, deflate',
        'Host': 'www.sec.gov'
    }
    
    # Format CIK to 10 digits
    cik_padded = str(cik).zfill(10)
    url = f'https://data.sec.gov/submissions/CIK{cik_padded}.json'
    
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            tickers = data.get('tickers', [])
            ticker = tickers[0] if tickers else ''
            return {
                'sic': data.get('sicCode', ''),
                'sic_description': data.get('sicDescription', ''),
                'ticker': ticker
            }
    except Exception as e:
        print(f"Error fetching company info for CIK {cik}: {str(e)}")
    return None

def process_delistings(df):
    """
    Process delisting data and add ticker symbols and SIC information
    """
    # Add new columns
    df['ticker'] = None
    df['sic'] = None
    df['sic_description'] = None
    
    # Process each company
    for idx, row in df.iterrows():
        company_info = get_company_info(row['cik'])
        if company_info:
            df.at[idx, 'ticker'] = company_info['ticker']
            df.at[idx, 'sic'] = company_info['sic']
            df.at[idx, 'sic_description'] = company_info['sic_description']
        time.sleep(0.1)  # Respect SEC API rate limits
    
    # Filter for biotech companies (SIC code 2836)
    biotech_companies = df[df['sic'] == '2836'].copy()
    
    return df, biotech_companies

# def export_results(df, biotech_df, output_file):
#     """
#     Export results to Excel with multiple sheets
#     """
#     with pd.ExcelWriter(output_file) as writer:
#         df.to_excel(writer, sheet_name='All Delistings', index=False)
#         biotech_df.to_excel(writer, sheet_name='Biotech Companies', index=False)

def main():
    # Read the input data
    dee_l_df = read_delisting_data('/Users/sudz4/Desktop/SPS_local/sps/x_bio_weapon/all_delistings_since_2020.csv')

    # Process the data
    processed_df, biotech_df = process_delistings(dee_l_df)

    # # Export results
    # export_results(processed_df, biotech_df, 'delisting_analysis.xlsx')

    # Print summary statistics
    print(f"Total delistings: {len(processed_df)}")
    print(f"Biotech companies: {len(biotech_df)}")
    print(f"Percentage biotech: {(len(biotech_df) / len(processed_df)) * 100:.1f}%")

if __name__ == "__main__":
    main()